<a href="https://colab.research.google.com/github/gustavoalcantara/capim/blob/main/capim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Case Data Analyst - Capim**\
**Gustavo Alcântara**\
**São José dos Campos, SP, Brasil**\
**Foco: Compreender o período de trial e como ele pode ser melhorado.**

Este Case faz parte da etapa do processo seletivo para a vaga de Data Anaylist da [Capim](https://capim.com.br/site). Todos os códigos/gráficos podem ser acessados via este colab e, caso necessário, replicados em seu ambiente. Verifique a necessidade dos arquivos utilizados estarem em seu Local.


Importe as bibliotecas

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as px
import plotly.express as px

Leia os arquivos localmente

In [39]:
#Reading the csvs archives
activity = pd.read_csv('activity.csv')
clinics = pd.read_csv('clinics.csv')
subscriptions = pd.read_csv('subscriptions.csv')

Compreensão das tabelas e variáveis

A tabela `activity` possui 12 colunas, tendo um total de 13941 observações. Neste contexto, essa tabela traz o Id de cada clinica atendida no período de Agosto a Setembro de 2023 e a interação de cada uma das clinicas com as variáveis. Ao total, são `1070` clinicas diferentes que estão nesse dataframe. O arquivo `clinics` possui 2219 observações que contempla um total de 2219 clinicas diferentes. Portanto, ao realizar uma junção interna (inner join) entre essas duas ou mais tabelas no futuro, apenas as clínicas presentes em ambos os conjuntos de dados serão consideradas.

Pensando em `um Dashboard operacional para o time de Produto - Aquisição acompanhar este processo, com gráficos e números gerais a serem monitorados diariamente e semanalmente` optei por apresentar um gráfico da interação das clinicas com a plataforma a partir da variável `ACTIVITY_TYPE` no período agosto a setembro de 2023. É possível observar que as atividades que mais ocorrem é a de `criação de pacientes` e `procedimentos criados`. É válido refletir que a plataforma, de fato, tem sua utilização para monitoramento e SaaS para gestão odontólógica.

In [59]:
subscriptions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 18 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   CLINIC_ID                               385 non-null    int64  
 1   STRIPE_SUBSCRIPTION_ID                  385 non-null    object 
 2   SUBSCRIPTION_START_DATE                 384 non-null    object 
 3   SUBSCRIPTION_END_DATE                   46 non-null     object 
 4   SUBSCRIPTION_CURRENT_PERIOD_STARTED_AT  385 non-null    object 
 5   SUBSCRIPTION_CURRENT_PERIOD_ENDS_AT     385 non-null    object 
 6   MOST_RECENT_INVOICE_CREATED_AT          385 non-null    object 
 7   LAST_PAYMENT_AT                         371 non-null    object 
 8   CHECKOUT_STATUS                         383 non-null    object 
 9   SUBSCRIPTION_STATUS                     383 non-null    object 
 10  BILLING_INTERVAL_IN_MONTHS              384 non-null    float6

In [49]:
(activity['ACTIVITY_TYPE'].value_counts()
 .reset_index()
 .pipe((px.bar, 'data_frame'), x='index', y='ACTIVITY_TYPE',
       labels={'index':'Tipo de Atividade', 'ACTIVITY_TYPE':'Contagem'},
       title='Contagem de Cada Tipo de Atividade no periodo de Ago. a Set. 2023'))

Mas, é necessário ter em mente como o periódo de trial pode ser melhorado. Diante disso, optei por verificar se as clinicas `Independente`, `Independente - Legado` e `Rede Homologada` possuem diferenças na duração de tempo de Trial. É fato que a diferença da quantidade de clinicas pode interferir neste valor. Porém, é perceptivel que o número de clinicas indepentes é o que possui a maior média de tempo de Trial. Será que essas clínicas possuem caracteristicas e interesses diferentes na utilização da plataforma?

In [55]:
(y.groupby('BUSINESS_SEGMENTATION')
   .agg({'CLINIC_ID':'count', 'TRIAL_DURATION':'mean'})
   .rename(columns={'CLINIC_ID':'Número de Clínicas',
                    'TRIAL_DURATION':
                    'Duração Média do Trial'}))

,Número de Clínicas,Duração Média do Trial
BUSINESS_SEGMENTATION,,
Independente,2055,8.474453
Independente - Legado,21,8.428571
Rede Homologada,143,7.146853


Assim, realizei um `

In [61]:
percentage_valid_activities = (
    pd.merge(activity, clinics, on='CLINIC_ID')
    .assign(is_valid_activity=lambda x: x['IS_VALID_ACTIVITY'].astype(int))
    .groupby('BUSINESS_SEGMENTATION')
    .agg(percentage_valid_activities=('is_valid_activity', 'mean'))
    .mean()['percentage_valid_activities'] * 100
)
percentage_valid_activities

24.71895875159276

In [18]:
data['STATUS'] = 'trial'
data.loc[data['SUBSCRIPTION_START_DATE'].notna(), 'STATUS'] = 'subscriber'

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2221 entries, 0 to 2220
Data columns (total 50 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   CLINIC_ID                                   2221 non-null   int64         
 1   CLINIC_CREATED_AT                           2221 non-null   datetime64[ns]
 2   BUSINESS_SEGMENTATION                       2221 non-null   object        
 3   IS_CHAIN_CLINIC                             2221 non-null   bool          
 4   FIRST_USER_HUBSPOT_SOURCE                   2221 non-null   object        
 5   INTEREST_REASONS                            2068 non-null   object        
 6   INTEREST_CATEGORY_SIGNUP                    2221 non-null   object        
 7   HAS_INTEREST_BNPL                           2221 non-null   bool          
 8   HAS_INTEREST_BNPL_ONLY                      2221 non-null   bool          
 9   HAS_INTE

In [25]:
status_counts = data['STATUS'].value_counts()

# Criar um DataFrame para o gráfico
df = pd.DataFrame({'Status': status_counts.index, 'Número de Clínicas': status_counts.values})

# Criar um gráfico de barras
fig = px.bar(df, x='Status', y='Número de Clínicas', color='Status', title='Número de Clínicas por Status')
fig.show()

In [ ]:
y['CLINIC_ID'].nunique()

2219